In [ ]:
import os
open_api_key='Provide open_api_key'
os.environ['OPENAI_API_KEY']=open_api_key

In [ ]:
# Basic prompt summarization

from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
speech='''Today, as we gather to reflect on our journey and look towards the future, I am filled with immense pride and gratitude for the progress we have made together.Our nation, rich in history and diverse in culture, stands as a beacon of hope and resilience.'''

In [ ]:
# This is the 1st type of text summarization technique
chat_messages=[
    SystemMessage(content="You are an expert assistant with expertize in summarization speeches"),
    HumanMessage(content="Please provide and short and concise summary of the following speech")
]

llm=ChatOpenAI(model_name='gpt-3.5-turbo')

In [ ]:
# total tokens
llm.get_num_tokens(speech)

In [ ]:
# get summary
print(llm(chat_messages).content)

**Prompt templates for text summarization**

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [ ]:
generic_template='''
Write the summary of the following speech:
Speech:{speech}
Translate the precise summary to {language}
'''


prompt=PromptTemplate(
    input_variables=['speech','language'],
    template=generic_template
)

In [ ]:
complete_prompt=prompt.format(speech=speech,language='Hindi')

In [ ]:
llm.get_num_tokens(complete_prompt)

In [ ]:
llm_chain=LLMChain(llm=llm,prompt=prompt)
summary=llm_chain.run(speech=speech,language='Hindi')

In [ ]:
summary

**StuffDocumentChain text summarization**

In [ ]:
from PyPDF2 import PDFReader

In [ ]:
# provide the path of pdf files

pdfreader=PDFReader('apjspeech.pdf')

In [ ]:
from typing_extensions import Concatenate
# read text from pdf
text=''
for i,page in enumerate(pdfreader.pages):
  content=page.extract_text()
  if content:
      text+=page.extract

In [ ]:
text

In [ ]:
from langchain.docstore.document import Document

docs=[Document(page_content=text)] # for converting entire text in the form of the document
docs

In [ ]:
llm=ChatOpenAI(temperature=0,model_name='gpt-3.5-turbo')

In [ ]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.load.summarize import load_summarize_chain

In [ ]:
template='''Write a concise and short summary of the following speech:
Speech:{text}
'''
prompt=PromptTemplate(
    input_variables=['text'],
    template=template
)



chain=load_summarize_chain(
    llm,
    chain_type='stuff',
    prompt=prompt,
    verbose=False
)

output_summary=chain.run(docs)
output_summary

**Summarzing large documents using map reduce**

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter # for splitting text into chuncks


# provide the path of the file
pdfreader=PDFReader('apjspeech.pdf')
from typing_extensions import Concatenate
# read text from pdf

text=''
for i,page in enumerate(pdfreader.pages):
  content=page.extract_text()
  if content:
      text+=page.extract

In [ ]:
llm=ChatOpenAI(temperature=0,model_name='gpt-3.5-turbo')

In [ ]:
llm.get_num_tokens(text)

In [ ]:
# splitting the text

text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

In [ ]:
chunks=text_splitter.create_documents([text])

In [ ]:
chunks
len(chunks)

In [ ]:
chain=load_summarize_chain(
    llm,
    chain_type='map_reduce',
    verbose=False
)

summary=chain.run(chunks)
summary  # this technique is used when we have file with huge data

**Map reduce with custom prompts**

In [ ]:
chunks_prompts="""
Write a concise and short summary of the following speech:
Speech:{text}
Summary:
"""

map_prompt_template=PromptTemplate(
    input_variables=['text'],
    template=chunks_prompts
)

In [ ]:
final_combine_prompt="""
Provide the final summary of the entire speech with these important points.
Add a generc Motivational title,
Start the precise summary with an introduction and provide the summary
in number points of the speech.
Speech:{text}
"""

final_prompt_template=PromptTemplate(
    input_variables=['text'],
    template=final_combine_prompt
)

In [ ]:
summary_chain=load_summarize_chain(
    llm,
    chain_type='map_reduce',
    map_prompt=map_prompt_template,
    combine_prompt=final_prompt_template,
    verbose=False
)

output=summary_chain.run(chunks)
output